## Problem 3: How many people live near shopping centers? (8 points)

In the last step of this analysis, use a *spatial join* to relate data from a population grid data set to the buffer layer created in *problem 2* to find out how many people live in all population grid cells that are **within** 1.5 km distance from each shopping centre. 

Use the same population grid data set as during [lesson 3](https://autogis-site.readthedocs.io/en/latest/lessons/lesson-3/spatial-join.html) (load it directly from WFS, don’t forget to assign a CRS).


*Feel free to divide your solution into more codeblocks than prepared! Remember to add comments to your code :)*

### a) Load the population grid data set and the buffer geometries (2 points)

Use the same population grid data set as during [lesson 3](https://autogis-site.readthedocs.io/en/latest/lessons/lesson-3/spatial-join.html) (load it directly from WFS, don’t forget to assign a CRS). Load the data into a `GeoDataFrame` called `population_grid`.

(optional) If you want, discard unneeded columns and translate the remaining column names from Finnish to English.

In [30]:
# ADD YOUR OWN CODE HERE
import geopandas
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

population_grid = geopandas.read_file(
    (
        "https://kartta.hsy.fi/geoserver/wfs"
        "?service=wfs"
        "&version=2.0.0"
        "&request=GetFeature"
        "&typeName=asuminen_ja_maankaytto:Vaestotietoruudukko_2020"
        "&srsName=EPSG:3879"
    ),
)
population_grid.crs = "EPSG:3879" 

In [5]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION
import geopandas
import pyproj

assert isinstance(population_grid, geopandas.GeoDataFrame)
assert population_grid.crs == pyproj.CRS("EPSG:3879")



Load the buffers computed in *problem 2* into a `GeoDataFrame` called `shopping_centre_buffers`. Add an `assert` statement to check whether the two data frames are in the same CRS.

In [65]:
# ADD YOUR OWN CODE HERE
shopping_centre_buffers = geopandas.read_file('shopping_centres.gpkg', layer='buffers')

In [10]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION
assert isinstance(shopping_centre_buffers, geopandas.GeoDataFrame)
assert shopping_centre_buffers.geometry.geom_type.unique() == ["Polygon"]
assert shopping_centre_buffers.crs == pyproj.CRS("EPSG:3879")


---

### b) Carry out a *spatial join* between the `population_grid` and the `shopping_centre_buffers`  (2 points)

Join the shopping centre’s `id` column (and others, if you want) to the population grid data frame, for all population grid cells that are **within** the buffer area of each shopping centre. [Use a *join-type* that retains only rows from both input data frames for which the geometric predicate is true](https://geopandas.org/en/stable/gallery/spatial_joins.html#Types-of-spatial-joins). 


In [66]:
# ADD YOUR OWN CODE HERE
# population_grid = population_grid[["asukkaita", "geometry"]]
population_grid = population_grid.rename(columns={"asukkaita": "population"})

population_grid["population_density"] = (
    population_grid["population"]
    / (population_grid.area / 1_000_000)
)

In [67]:
shopping_centres_with_population = shopping_centre_buffers.sjoin(
    population_grid,
    how='left',
    predicate='within'
)


---

### c) Compute the population sum around shopping centres (2 points)

Group the resulting (joint) data frame by shopping centre (`id` or `name`), and calculate the `sum()` of the population living inside the 1.5 km radius around them.

Print the results, for instance, in the form "12345 people live within 1.5 km from REDI".

In [68]:
shopping_centres_with_population

,address,id,name,addr,geometry,index_right0,gml_id,index,population,asvaljyys,ika0_9,ika10_19,ika20_29,ika30_39,ika40_49,ika50_59,ika60_69,ika70_79,ika_yli80,population_density
0,"Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...",0,Itis,"Itäkatu 1-7, 00930 Helsinki, Finland","POLYGON ((25504600.102 6677662.109, 25504600.0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Change Lingerie, 14-20, Mannerheimintie, Kesku...",1,Forum,"Mannerheimintie 14–20, 00100 Helsinki, Finland","POLYGON ((25496593.134 6672925.394, 25496593.1...",3438.0,Vaestotietoruudukko_2020.3439,16308.0,133.0,40.0,4.0,7.0,33.0,26.0,20.0,15.0,10.0,10.0,8.0,2128.027873
2,"Musti ja Mirri, 11, Piispansilta, Matinkylä, S...",2,Iso-omena,"Piispansilta 11, 02230 Espoo, Finland","POLYGON ((25485476.69 6671962.378, 25485476.68...",1379.0,Vaestotietoruudukko_2020.1380,9022.0,652.0,44.0,31.0,18.0,198.0,134.0,53.0,51.0,59.0,74.0,33.0,10432.136638
3,"Taco Bell, 3-9, Leppävaarankatu, Ruusutorppa, ...",3,Sello,"Leppävaarankatu 3-9, 02600 Espoo, Finland","POLYGON ((25489647.829 6678453.64, 25489647.82...",2207.0,Vaestotietoruudukko_2020.2208,11750.0,282.0,29.0,26.0,8.0,119.0,70.0,22.0,9.0,9.0,15.0,4.0,4512.059098
4,"Stockmann, 3, Vantaanportinkatu, Vantaanportti...",4,Jumbo,"Vantaanportinkatu 3, 01510 Vantaa, Finland","POLYGON ((25497945.432 6686656.982, 25497945.4...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"Valaskala, 5, Hermannin rantatie, Verkkosaari,...",5,REDI,"Hermannin rantatie 5, 00580 Helsinki, Finland","POLYGON ((25498990.071 6674918.261, 25498990.0...",3825.0,Vaestotietoruudukko_2020.3826,17758.0,647.0,33.0,31.0,31.0,261.0,137.0,58.0,67.0,41.0,19.0,2.0,10351.721438
6,"Mall of Tripla, 1, Fredikanterassi, Keski-Pasi...",6,Tripla,"Fredikanterassi 1, 00520 Helsinki, Finland","POLYGON ((25496147.158 6676164.227, 25496147.1...",3355.0,Vaestotietoruudukko_2020.3356,15971.0,534.0,40.0,22.0,17.0,169.0,172.0,49.0,62.0,30.0,10.0,3.0,8544.111909


In [69]:
# ADD YOUR OWN CODE HERE
for name, population in shopping_centres_with_population.groupby('name')['population_density'].sum().items():
    print(f'{population:.0f} people live within 1.5 km from {name}')

2128 people live within 1.5 km from Forum
10432 people live within 1.5 km from Iso-omena
0 people live within 1.5 km from Itis
0 people live within 1.5 km from Jumbo
10352 people live within 1.5 km from REDI
4512 people live within 1.5 km from Sello
8544 people live within 1.5 km from Tripla 



---

### d) Reflection

Good job! You are almost done with this week’s exercise. Please quickly answer the following short questions:
    
- How challenging did you find problems 1-3 (on scale to 1-5), and why?
- What was easy?
- What was difficult?

Add your answers in a new *Markdown* cell below: